In [1]:
import pystac_client
import planetary_computer
import odc.stac
import matplotlib.pyplot as plt
import stackstac
from pystac.extensions.eo import EOExtension as eo
import numpy as np
import math
import geopandas as gpd
import rasterio as rio
import rioxarray
from datetime import datetime

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [5]:
bbox_of_interest = [-110.0, 42.7, -109.0, 43.5]
time_of_interest = "2022-08-01/2022-10-01"

In [6]:
# search s2 catalog for scenes with low cloud cover
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)

items = search.item_collection()
print(f"Returned {len(items)} Items")

Returned 33 Items


In [7]:
# load into xarray ds
ds = (
    stackstac.stack(
        items,
        assets=["B04","B03","B02", "B08"],  # red, green, blue, nir
        chunksize=4096,
        bounds_latlon=bbox_of_interest
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata 
)

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [ ]:
# plot all scenes to determine which are best for feature tracking
f, ax = plt.subplots(len(ds.time), 2, figsize=(10, 5*len(ds.time)))
for i, time in enumerate(ds.time):
    ds.isel(time=i, band=[0, 1, 2]).plot.imshow(ax=ax[i, 0], rgb='band', robust=True)
    ax[i, 0].set_title(f'time: {time.dt.date.item()}, index: {i}, rgb')
    ax[i, 0].set_aspect('equal')
    ax[i, 0].axis('off')
    
    ds.isel(time=i, band=3).plot.imshow(ax=ax[i, 1], add_colorbar=False)
    ax[i, 1].set_title('NIR')
    ax[i, 1].set_aspect('equal')
    ax[i, 1].axis('off')

In [ ]:
# select scenes, select nir band
img1 = ds.isel(time=2, band=3).rio.write_nodata(-9999).values

img2 = ds.isel(time=18, band=3).rio.write_nodata(-9999).values

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(img1)
ax[0].set_aspect('equal')
ax[0].set_title('S2 NIR 2022-09-19')
ax[1].imshow(img2)
ax[1].set_title('S2 NIR 2022-10-29')
ax[1].set_aspect('equal')
plt.tight_layout()

plt.savefig('./figs/karakoram_orthos_nir.png', dpi=300)

In [ ]:
## install autoRIFT in notebook environment
# import sys
# !mamba install --yes --prefix {sys.prefix} autorift
# !{sys.executable} -m pip install opencv-python-headless

In [ ]:
from autoRIFT import autoRIFT

In [ ]:
# setup autoRIFT parameters
obj = autoRIFT()
obj.I1 = img1
obj.I2 = img2
    
# define chip size ranges
## Kernel sizes to use for correlation
obj.ChipSizeMinX = 16
obj.ChipSizeMaxX = 128

# check and refine this
obj.ChipSize0X = 16

# skip rate setup
# Produce output at a grid x time the input resolution 
obj.SkipSampleX = 4
obj.SkipSampleY = 4
# here selecting 4 means our output of 40 m resolution, correlation result at every 4th index on grid

# preprocess to bring out edges in the image
prefilter_choice = 'lap'
walis_filter_width = 3
if not prefilter_choice == 'none':
    obj.WallisFilterWidth = walis_filter_width
    if prefilter_choice == 'db':
        print(f"####### Using DB preprocessing with a window width of {walis_filter_width} #######")
        obj.preprocess_db()
    elif prefilter_choice == 'lap':
        print(f"####### Using a laplician filter with a window width of {walis_filter_width} #######")
        obj.preprocess_filt_lap()
    elif prefilter_choice == 'sob':
        print(f"####### Using a Sobel filter with a window width of {walis_filter_width} #######")
        obj.preprocess_filt_sob()
    elif prefilter_choice == 'wal':
        print(f"####### Using a Walis filter with a window width of {walis_filter_width} #######")
        obj.preprocess_filt_wal()
else:
    print("##### No preprocessing #####")
    
convert8bit = True
# converting to 8 bit and then performing correlation is a faster approach
print(f"convert8bit option is {convert8bit}")
if convert8bit:
    print("####### Downcasting to 8 bit#######")
    obj.uniform_data_type()

print("creating correlation grid")
# create the grid if it does not exist (from test_autoRIFT.py)
m,n = obj.I1.shape
xGrid = np.arange(obj.SkipSampleX+10,n-obj.SkipSampleX,obj.SkipSampleX)
yGrid = np.arange(obj.SkipSampleY+10,m-obj.SkipSampleY,obj.SkipSampleY) 
nd = xGrid.__len__()
md = yGrid.__len__()
obj.xGrid = np.int32(np.dot(np.ones((md,1)),np.reshape(xGrid,(1,xGrid.__len__()))))
obj.yGrid = np.int32(np.dot(np.reshape(yGrid,(yGrid.__len__(),1)),np.ones((1,nd))))
noDataMask = np.logical_not(obj.xGrid)

In [ ]:
# run autorift
print("####### Executing autoRIFT correlation #######")
obj.runAutorift()

In [ ]:
f,ax = plt.subplots(1,2, figsize=(10, 5))
im = ax[0].imshow(obj.Dx, cmap='RdBu', vmin=-5, vmax=5)
ax[0].set_title('x offsets')
ax[0].set_aspect('equal')
im = ax[1].imshow(obj.Dy ,cmap='RdBu', vmin=-5, vmax=5)
ax[1].set_title('y offsets')
ax[1].set_aspect('equal')
plt.colorbar(im, label='pixels')
plt.tight_layout()

plt.savefig('./figs/karakoram_xyoffsets.png', dpi=300)

In [ ]:
vm = np.ma.sqrt(obj.Dx**2+obj.Dy**2)

In [ ]:
t1 = datetime(2022, 9, 19)
t2 = datetime(2022,10,29)
dt_year = (t2-t1).days/365.25

In [ ]:
xres,yres = (10,10) # input resolution of Sentinel-2 images

In [ ]:
def disp2v(dx,dy,xres,yres,t_factor,dt='yr'):
    """
    Inspired from vmap's functionality
    """
    if dt == 'day':
        t_factor = t_factor*365.25
    if dt != 'px':
        h_myr = (dx * xres)/t_factor
        v_myr = (dy * yres)/t_factor
    else:
        h_myr = dx
        v_myr = dy
        t_factor = 1
    vm_myr = np.ma.sqrt(h_myr**2+v_myr**2)
    return(h_myr,v_myr,vm_myr)

In [ ]:
vx_myr,vy_myr,vm_myr = disp2v(obj.Dx,obj.Dy,xres,yres,dt_year)

In [ ]:
f,ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(vm_myr,vmin=0,vmax=500,cmap='magma')
ax.set_aspect('equal')
plt.colorbar(im, label='m/yr')
ax.set_title("mean velocity, 2022-09-19 to 2022-10-29")
#plt.savefig('./figs/karakoram_veloc.png', dpi=300)

In [ ]:
f,ax = plt.subplots()
ax.imshow(obj.InterpMask,cmap='gray')
ax.set_title('Interpolation map')
# these pixels are the ones which were interpolated, not correlated by feature tracking

In [ ]:
f,ax = plt.subplots()
im = ax.imshow(obj.ChipSizeX,cmap='Accent')
plt.colorbar(im)
ax.set_title('Final Chip size used')
# chip sizes of this length were used in correlation

In [ ]:
fake_init_gt = [519260.0,10,0.0,4030010.0,0.0,-10.0,]
ulx = fake_init_gt[0]+obj.xGrid[0,0]*fake_init_gt[1] # upper-left cordinate
resx = fake_init_gt[1]*obj.SkipSampleX # horizontal resolution
rotx = fake_init_gt[2] # no-change
uly = fake_init_gt[3] + obj.yGrid[0,0]*fake_init_gt[5]
roty = fake_init_gt[4]
resy = fake_init_gt[5]*obj.SkipSampleY
gt_fn = [ulx,resx,rotx,uly,roty,resy]

In [ ]:
def gdal2rasterio_transform(gt_fn):
    return rio.Affine(gt_fn[1],gt_fn[2],gt_fn[0],gt_fn[4],gt_fn[5],gt_fn[3])

In [ ]:
def ar2tif(ma_ar,outfn,crs,nodata,transform):
    with rio.open(outfn,
                      'w',
                      driver='GTiff',
                      height=ma_ar.shape[0],
                      width=ma_ar.shape[1],
                      count=1,
                      dtype=ma_ar.dtype,
                      crs=crs,
                      nodata=nodata,
                      transform=transform) as dst:
        dst.write(np.ma.filled(ma_ar,fill_value=nodata),1)

In [ ]:
# # save the files one by one
# ar2tif(vx_myr,'20220919_20221029_EW_velocity_myr.tif',
#       ds.crs,-9999.0,gdal2rasterio_transform(gt_fn))